In [1]:
import pandas as pd
import os
import detect_simpsons_paradox as dsp
import numpy as np

We'll first load in some data, this has both regression and rate type trends, since this file has a weird index, we'll load it in as dataframe first

In [2]:
hit_search_rate = pd.read_csv('../data/state_hit_rate_min_cols_COCTFLILMDMAMOMTNENCOHRISCTXVTWAWI.csv',index_col='Unnamed: 0')
hit_search_rate.head()

,state,year,driver_gender,driver_race,decriminalization,medical,recreational,no_reforms,search_conducted_false,search_conducted_true,search_conducted_rate,contraband_found_false,contraband_found_true,contraband_found_rate,hit_false,hit_true,hit_rate,num_stops
0,AZ,2009,F,White,0,0,0,1,30.0,1.0,0.032258,31.0,NaN,NaN,1.0,NaN,NaN,31.0
1,AZ,2009,M,Black,0,0,0,1,1.0,2.0,0.666667,2.0,1.0,0.333333,1.0,1.0,0.5,3.0
2,AZ,2009,M,Hispanic,0,0,0,1,14.0,1.0,0.066667,15.0,NaN,NaN,1.0,NaN,NaN,15.0
3,AZ,2009,M,White,0,0,0,1,32.0,4.0,0.111111,34.0,2.0,0.055556,2.0,2.0,0.5,36.0
4,AZ,2010,F,Asian,0,1,0,0,58.0,3.0,0.049180,61.0,NaN,NaN,3.0,NaN,NaN,61.0


We can now use the labeledDataFrame with that DataFrame to create the object

In [3]:
labeled_df_setup = dsp.labeledDataFrame(hit_search_rate)

Next, we can infer the variable types

In [4]:
labeled_df_setup.infer_var_types()
labeled_df_setup.meta_df.head()

,dtype,var_type,role,isCount,weighting_var
variable,,,,,
state,object,categorical,NaN,NaN,NaN
year,int64,ordinal,NaN,NaN,NaN
driver_gender,object,binary,NaN,NaN,NaN
driver_race,object,categorical,NaN,NaN,NaN
decriminalization,int64,binary,NaN,NaN,NaN


For this, we'll manually set these, but in the vizualization tool you can also set these with drop down menus

In [5]:
roles = {'state':['trend','groupby'], 'year':'trend', 'driver_gender':['trend','groupby'], 'driver_race':['trend','groupby'],
       'decriminalization':['groupby'], 'medical':['groupby'],
         'recreational':['groupby'], 'no_reforms':['groupby'],
       'search_conducted_false':'ignore', 'search_conducted_true':'ignore',
       'search_conducted_rate':'trend', 'contraband_found_false':'ignore',
       'contraband_found_true':'ignore', 'contraband_found_rate':'trend', 'hit_false':'ignore',
       'hit_true':'ignore', 'hit_rate':'trend', 'num_stops':'trend'}
is_count = {'state':False, 'year':False, 'driver_gender':False, 'driver_race':False,
       'decriminalization':False, 'medical':False, 'recreational':False, 'no_reforms':False,
       'search_conducted_false':True, 'search_conducted_true':True,
       'search_conducted_rate':False, 'contraband_found_false':True,
       'contraband_found_true':True, 'contraband_found_rate':False, 'hit_false':True,
       'hit_true':True, 'hit_rate':False, 'num_stops':True}
count_list = ['search_conducted_false', 'search_conducted_true','contraband_found_false',
       'contraband_found_true', 'hit_false',
       'hit_true', 'num_stops']
var_types = {'driver_gender':'categorical'}
weighting = {'hit_rate':'search_conducted_true','search_conducted_rate':'num_stops','contraband_found_rate':'num_stops'}

We'll set those next.  Above gives examples of two ways that we can specify the count values to pass them to the set_counts function, but we'll only call it once below. 

In [6]:
labeled_df_setup.set_counts(count_list)
labeled_df_setup.set_roles(roles)
labeled_df_setup.set_weighting_vars(weighting)
labeled_df_setup.set_var_types(var_types)
labeled_df_setup.meta_df

/home/smb/anaconda3/lib/python3.6/site-packages/detect_simpsons_paradox/labeled_dataframe.py:223: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.meta_df['role'][k] = v
/home/smb/anaconda3/lib/python3.6/site-packages/detect_simpsons_paradox/labeled_dataframe.py:207: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.meta_df['var_type'][k] = v


,dtype,var_type,role,isCount,weighting_var
variable,,,,,
state,object,categorical,"[trend, groupby]",False,NaN
year,int64,ordinal,trend,False,NaN
driver_gender,object,categorical,"[trend, groupby]",False,NaN
driver_race,object,categorical,"[trend, groupby]",False,NaN
decriminalization,int64,binary,[groupby],False,NaN
medical,int64,binary,[groupby],False,NaN
recreational,int64,binary,[groupby],False,NaN
no_reforms,int64,binary,[groupby],False,NaN
search_conducted_false,float64,continuous,ignore,True,NaN


Now, we've set this up, we can also save these configurations to load them in directly in the future

In [7]:
labeled_df_setup.to_csvs('../data/ldf_state_hit_rate_min_cols_COCTFLILMDMAMOMTNENCOHRISCTXVTWAWI')

We can see what this does, using a bash magic

In [8]:
%%bash
cd ../data/ldf_state_hit_rate_min_cols_COCTFLILMDMAMOMTNENCOHRISCTXVTWAWI
ls

df.csv
meta.csv
result_df.csv


it write the three DataFrames each out to their own .csv file in that directory. If that directory exists it will overwrite without warning, if not, also creates the directory. 

Now, we can can also load the data back

In [9]:
labeled_df = dsp.labeledDataFrame('../data/ldf_state_hit_rate_min_cols_COCTFLILMDMAMOMTNENCOHRISCTXVTWAWI')
labeled_df.meta_df

,dtype,var_type,role,isCount,weighting_var
variable,,,,,
state,object,categorical,"[trend, groupby]",False,NaN
year,int64,ordinal,[trend],False,NaN
driver_gender,object,categorical,"[trend, groupby]",False,NaN
driver_race,object,categorical,"[trend, groupby]",False,NaN
decriminalization,int64,binary,[groupby],False,NaN
medical,int64,binary,[groupby],False,NaN
recreational,int64,binary,[groupby],False,NaN
no_reforms,int64,binary,[groupby],False,NaN
search_conducted_false,float64,continuous,[ignore],True,NaN


In [10]:
labeled_df.add_all_dpgmm(qual_thresh =.2)
labeled_df.df.head()

/home/smb/anaconda3/lib/python3.6/site-packages/sklearn/mixture/base.py:237: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  % (init + 1), ConvergenceWarning)


,state,year,driver_gender,driver_race,decriminalization,medical,recreational,no_reforms,search_conducted_false,search_conducted_true,...,hit_true,hit_rate,num_stops,search_conducted_false_search_conducted_true_dpgmm,search_conducted_false_search_conducted_rate_dpgmm,search_conducted_false_contraband_found_false_dpgmm,search_conducted_false_num_stops_dpgmm,search_conducted_true_contraband_found_false_dpgmm,search_conducted_true_num_stops_dpgmm,contraband_found_false_num_stops_dpgmm
0,AZ,2009,F,White,0,0,0,1,30.0,1.0,...,NaN,NaN,31.0,0,0,3,11,0,9,7
1,AZ,2009,M,Black,0,0,0,1,1.0,2.0,...,1.0,0.5,3.0,0,6,3,11,0,9,7
2,AZ,2009,M,Hispanic,0,0,0,1,14.0,1.0,...,NaN,NaN,15.0,0,0,3,11,0,9,7
3,AZ,2009,M,White,0,0,0,1,32.0,4.0,...,2.0,0.5,36.0,0,6,3,11,0,9,7
4,AZ,2010,F,Asian,0,1,0,0,58.0,3.0,...,NaN,NaN,61.0,0,0,3,11,0,9,7


In [11]:
labeled_df.meta_df

,dtype,var_type,role,isCount,weighting_var
state,object,categorical,"[trend, groupby]",False,NaN
year,int64,ordinal,[trend],False,NaN
driver_gender,object,categorical,"[trend, groupby]",False,NaN
driver_race,object,categorical,"[trend, groupby]",False,NaN
decriminalization,int64,binary,[groupby],False,NaN
medical,int64,binary,[groupby],False,NaN
recreational,int64,binary,[groupby],False,NaN
no_reforms,int64,binary,[groupby],False,NaN
search_conducted_false,float64,continuous,[ignore],True,NaN
search_conducted_true,float64,continuous,[ignore],True,NaN


In [12]:
labeled_df.add_quantile(['hit_rate','num_stops'])

,state,year,driver_gender,driver_race,decriminalization,medical,recreational,no_reforms,search_conducted_false,search_conducted_true,...,num_stops,search_conducted_false_search_conducted_true_dpgmm,search_conducted_false_search_conducted_rate_dpgmm,search_conducted_false_contraband_found_false_dpgmm,search_conducted_false_num_stops_dpgmm,search_conducted_true_contraband_found_false_dpgmm,search_conducted_true_num_stops_dpgmm,contraband_found_false_num_stops_dpgmm,hit_ratequantiles,num_stopsquantiles
0,AZ,2009,F,White,0,0,0,1,30.0,1.0,...,31.0,0,0,3,11,0,9,7,high,low
1,AZ,2009,M,Black,0,0,0,1,1.0,2.0,...,3.0,0,6,3,11,0,9,7,high,low
2,AZ,2009,M,Hispanic,0,0,0,1,14.0,1.0,...,15.0,0,0,3,11,0,9,7,high,low
3,AZ,2009,M,White,0,0,0,1,32.0,4.0,...,36.0,0,6,3,11,0,9,7,high,low
4,AZ,2010,F,Asian,0,1,0,0,58.0,3.0,...,61.0,0,0,3,11,0,9,7,high,low
5,AZ,2010,F,Black,0,1,0,0,224.0,8.0,...,232.0,0,0,3,11,0,9,7,high,low
6,AZ,2010,F,Hispanic,0,1,0,0,557.0,33.0,...,590.0,0,0,3,11,0,9,7,low,low
7,AZ,2010,F,Other,0,1,0,0,167.0,6.0,...,173.0,0,0,3,11,0,9,7,high,low
8,AZ,2010,F,White,0,1,0,0,3145.0,92.0,...,3237.0,0,0,3,11,0,9,7,low,mid
9,AZ,2010,M,Asian,0,1,0,0,97.0,1.0,...,98.0,0,0,3,11,0,9,7,high,low


In [13]:
labeled_df.meta_df

,dtype,var_type,role,isCount,weighting_var
state,object,categorical,"[trend, groupby]",False,NaN
year,int64,ordinal,[trend],False,NaN
driver_gender,object,categorical,"[trend, groupby]",False,NaN
driver_race,object,categorical,"[trend, groupby]",False,NaN
decriminalization,int64,binary,[groupby],False,NaN
medical,int64,binary,[groupby],False,NaN
recreational,int64,binary,[groupby],False,NaN
no_reforms,int64,binary,[groupby],False,NaN
search_conducted_false,float64,continuous,[ignore],True,NaN
search_conducted_true,float64,continuous,[ignore],True,NaN


# Using Trends

Trend objects define their name, how to compute the trend and how to choose which variables, 

extension will allow that the var lists may be passed to reduce which ones are computed

In [14]:
corrobj = dsp.all_pearson()
corrobj.get_trend_vars(labeled_df)
corrobj.regression_vars

['year',
 'search_conducted_rate',
 'contraband_found_rate',
 'hit_rate',
 'num_stops']

In [15]:
rankobj = dsp.mean_rank_trend()
linreg_obj = dsp.all_linear_trend()

# Computing Trends on a labeledDataFrame

There are two ways, we can use default setting and pass the names of the trend type or a trend object

In [16]:
labeled_df.get_subgroup_trends_1lev(['pearson_corr'])
labeled_df.result_df.head()

,feat1,feat2,subgroup_trend,subgroup_trend_quality,group_feat,subgroup,trend_type,agg_trend,agg_trend_quality
0,year,search_conducted_rate,-0.247018,-0.247018,state,AZ,pearson_corr,-0.03903,-0.03903
1,year,search_conducted_rate,-0.414566,-0.414566,state,CO,pearson_corr,-0.03903,-0.03903
2,year,search_conducted_rate,0.118238,0.118238,state,CT,pearson_corr,-0.03903,-0.03903
3,year,search_conducted_rate,-0.199765,-0.199765,state,IL,pearson_corr,-0.03903,-0.03903
4,year,search_conducted_rate,-0.603026,-0.603026,state,MA,pearson_corr,-0.03903,-0.03903


Now we can use a list of objects and apply multiple trends

In [17]:
labeled_df.get_subgroup_trends_1lev([rankobj,linreg_obj])
labeled_df.result_df.sample(10)

/home/smb/anaconda3/lib/python3.6/site-packages/scipy/stats/_stats_mstats_common.py:107: RuntimeWarning: invalid value encountered in double_scalars
  slope = r_num / ssxm
/home/smb/anaconda3/lib/python3.6/site-packages/scipy/stats/_stats_mstats_common.py:117: RuntimeWarning: invalid value encountered in sqrt
  t = r * np.sqrt(df / ((1.0 - r + TINY)*(1.0 + r + TINY)))
/home/smb/anaconda3/lib/python3.6/site-packages/scipy/stats/_stats_mstats_common.py:119: RuntimeWarning: invalid value encountered in double_scalars
  sterrest = np.sqrt((1 - r**2) * ssym / ssxm / df)


,feat1,feat2,group_feat,subgroup,subgroup_trend,subgroup_trend_quality,trend_type,agg_trend,agg_trend_quality
1223,num_stops,state,search_conducted_true_contraband_found_false_d...,11,"[AZ, CT, RI, SC, IL, WA, TX, MD, VT, MA, CO, N...",0.166500,rank_trend,"[RI, VT, WI, CT, MD, MA, AZ, CO, IL, NC, WA, S...",0.320900
1585,year,num_stops,state,AZ,8474.29,0.301916,lin_reg,-1560.86,0.043965
1940,search_conducted_rate,num_stops,medical,0,-245124,0.056923,lin_reg,-257201,0.065594
908,search_conducted_rate,state,search_conducted_false_num_stops_dpgmm,2,"[WA, IL, MA, SC, TX, AZ]",0.516300,rank_trend,"[CO, NC, VT, WI, MD, MA, CT, TX, SC, WA, RI, I...",0.388100
2001,search_conducted_rate,num_stops,search_conducted_true_num_stops_dpgmm,0,-281701,0.652256,lin_reg,-257201,0.065594
622,hit_rate,driver_race,search_conducted_true_num_stops_dpgmm,5,"[Hispanic, Black, White]",0.678400,rank_trend,"[Asian, Hispanic, Other, Black, White]",0.002300
1300,year,search_conducted_rate,search_conducted_false_contraband_found_false_...,1,-0.000412834,1.000000,lin_reg,-0.000353381,0.039030
422,contraband_found_rate,driver_race,contraband_found_false_num_stops_dpgmm,8,[White],NaN,rank_trend,"[Asian, White, Other, Hispanic, Black]",0.071600
177,search_conducted_rate,driver_race,search_conducted_true_contraband_found_false_d...,2,[White],NaN,rank_trend,"[Asian, White, Other, Black, Hispanic]",0.277200
374,contraband_found_rate,driver_race,search_conducted_false_contraband_found_false_...,8,"[White, Hispanic, Black]",0.470300,rank_trend,"[Asian, White, Other, Hispanic, Black]",0.071600


These two methods give the same, the string based version allows for simple access to default setting but passing a trend object would allow for overriding defaults and creating more custom subests of trends.

We can see what types of trends were computed from `result_df`

In [18]:
pd.unique(labeled_df.result_df['trend_type'])

array(['rank_trend', 'lin_reg'], dtype=object)

The object also stores the trend objects that have been applied, they can be used for mapping to get the distance functions that are appropriate for each trend

In [19]:
labeled_df.trend_list

In [20]:
# labeled_df.result_df['distance'] = labeled_df.result_df.apply(dist_helper,axis=1)
labeled_df.add_distance()

labeled_df.result_df.sample(10)

/home/smb/anaconda3/lib/python3.6/site-packages/scipy/stats/stats.py:3559: RuntimeWarning: invalid value encountered in double_scalars
  size * (size - 1) * (size - 2))


,feat1,feat2,group_feat,subgroup,subgroup_trend,subgroup_trend_quality,trend_type,agg_trend,agg_trend_quality,distance
80,search_conducted_rate,driver_gender,search_conducted_true_contraband_found_false_d...,11,"[F, M]",0.126500,rank_trend,"[F, M]",0.251900,0.000000
1995,search_conducted_rate,num_stops,search_conducted_true_contraband_found_false_d...,6,-414660,0.652985,lin_reg,-257201,0.065594,0.000085
476,hit_rate,driver_gender,search_conducted_false_contraband_found_false_...,1,[M],NaN,rank_trend,"[F, M]",0.043300,NaN
1101,hit_rate,state,search_conducted_false_contraband_found_false_...,11,"[WA, NC, MD, SC, IL, AZ, CT, TX, RI, CO, MA, W...",0.548200,rank_trend,"[WA, NC, IL, AZ, MD, SC, TX, CT, RI, MA, WI, C...",0.221000,0.180000
470,hit_rate,driver_gender,search_conducted_false_search_conducted_rate_d...,2,"[M, F]",0.065200,rank_trend,"[F, M]",0.043300,2.000000
1694,year,num_stops,num_stopsquantiles,low,18.7416,0.099371,lin_reg,-1560.86,0.043965,176.909050
1237,num_stops,state,contraband_found_false_num_stops_dpgmm,1,"[SC, WA, TX]",0.454500,rank_trend,"[RI, VT, WI, CT, MD, MA, AZ, CO, IL, NC, WA, S...",0.320900,0.670000
1230,num_stops,state,search_conducted_true_num_stops_dpgmm,6,"[TX, WA]",0.550000,rank_trend,"[RI, VT, WI, CT, MD, MA, AZ, CO, IL, NC, WA, S...",0.320900,0.510000
1586,year,num_stops,state,CO,-5034.01,0.166912,lin_reg,-1560.86,0.043965,0.025326
762,num_stops,driver_race,state,SC,"[Other, Hispanic, Black, White]",0.791700,rank_trend,"[Other, Asian, Black, Hispanic, White]",0.499600,0.260000


Each trend object has a trend_precompute dictionary as a property that stores the intermediate values (tables of the weighted rates for ranks and correlation matrices for pearson correlation, TODO: what do we need for linreg). These can be used in vizualization.

In [21]:
labeled_df.trend_list[0].trend_precompute

{'rank_trend_agg_trend_contraband_found_rate_driver_gender': driver_gender
 F    0.003184
 M    0.006874
 dtype: float64,
 'rank_trend_agg_trend_contraband_found_rate_driver_race': driver_race
 Asian       0.002592
 White       0.004982
 Other       0.005700
 Hispanic    0.006455
 Black       0.008367
 dtype: float64,
 'rank_trend_agg_trend_contraband_found_rate_state': state
 NC    0.001062
 CO    0.002303
 MD    0.003503
 WA    0.004353
 CT    0.005716
 SC    0.006382
 TX    0.006541
 WI    0.007934
 IL    0.008215
 MA    0.008255
 VT    0.009567
 AZ    0.009980
 RI    0.013719
 dtype: float64,
 'rank_trend_agg_trend_hit_rate_driver_gender': driver_gender
 F    0.243620
 M    0.256954
 dtype: float64,
 'rank_trend_agg_trend_hit_rate_driver_race': driver_race
 Asian       0.165508
 Hispanic    0.193965
 Other       0.195967
 Black       0.269228
 White       0.279382
 dtype: float64,
 'rank_trend_agg_trend_hit_rate_state': state
 WA    0.135670
 NC    0.146822
 IL    0.205232
 AZ    0

# Filtering

In [22]:
help(labeled_df.get_trend_rows)

Help on method get_trend_rows in module detect_simpsons_paradox.ranking_processing:

get_trend_rows(feat1=None, feat2=None, group_feat=None, subgroup=None) method of detect_simpsons_paradox.labeled_dataframe.labeledDataFrame instance
    return a row of result_df based on the specified values. returned rows
    meet provided criteria for all columns (and operator) and any one of the listed
    values for each column (or operator)
    
    Parameters
    -----------
    feat1 : str, list, or  None
        trend variable name or None to include all
    feat2 : str, list, or  None
        trend variable name or None to include all
    group_feat : str, list, or  None
        groupoby variable name or None to include all
    subgroup : str, list, or  None
        value of groupby_feat or or None to include all



So, we can use that function to filter and look at subsets of the trends based on the features, groupby, or subgroups

In [23]:
labeled_df.get_trend_rows(feat1='year',subgroup=['Black','Hispanic'])

,feat1,feat2,group_feat,subgroup,subgroup_trend,subgroup_trend_quality,trend_type,agg_trend,agg_trend_quality,distance
1268,year,search_conducted_rate,driver_race,Black,-0.000555503,0.039174,lin_reg,-0.000353381,0.039030,0.011581
1269,year,search_conducted_rate,driver_race,Hispanic,-0.0014828,0.177694,lin_reg,-0.000353381,0.039030,0.064711
1601,year,num_stops,driver_race,Black,-2262.72,0.169689,lin_reg,-1560.86,0.043965,0.011386
1602,year,num_stops,driver_race,Hispanic,422.667,0.014732,lin_reg,-1560.86,0.043965,179.827735


We can also filter based on SP

In [24]:
labeled_df.get_SP_rows(thresh=.2)

,feat1,feat2,group_feat,subgroup,subgroup_trend,subgroup_trend_quality,trend_type,agg_trend,agg_trend_quality,distance,SP_thresh_0.2
36,search_conducted_rate,driver_gender,search_conducted_false_search_conducted_true_d...,10,"[M, F]",0.149400,rank_trend,"[F, M]",0.251900,2.000000,True
37,search_conducted_rate,driver_gender,search_conducted_false_search_conducted_true_d...,11,"[M, F]",0.881900,rank_trend,"[F, M]",0.251900,2.000000,True
43,search_conducted_rate,driver_gender,search_conducted_false_search_conducted_rate_d...,5,"[M, F]",0.126100,rank_trend,"[F, M]",0.251900,2.000000,True
44,search_conducted_rate,driver_gender,search_conducted_false_search_conducted_rate_d...,6,"[M, F]",NaN,rank_trend,"[F, M]",0.251900,2.000000,True
49,search_conducted_rate,driver_gender,search_conducted_false_contraband_found_false_...,4,"[M, F]",0.875000,rank_trend,"[F, M]",0.251900,2.000000,True
65,search_conducted_rate,driver_gender,search_conducted_false_num_stops_dpgmm,8,"[M, F]",0.130300,rank_trend,"[F, M]",0.251900,2.000000,True
67,search_conducted_rate,driver_gender,search_conducted_false_num_stops_dpgmm,10,"[M, F]",0.881900,rank_trend,"[F, M]",0.251900,2.000000,True
70,search_conducted_rate,driver_gender,search_conducted_true_contraband_found_false_d...,1,"[M, F]",0.881900,rank_trend,"[F, M]",0.251900,2.000000,True
77,search_conducted_rate,driver_gender,search_conducted_true_contraband_found_false_d...,8,"[M, F]",0.164600,rank_trend,"[F, M]",0.251900,2.000000,True
89,search_conducted_rate,driver_gender,search_conducted_true_num_stops_dpgmm,8,"[M, F]",0.881900,rank_trend,"[F, M]",0.251900,2.000000,True


# Ranking

In [25]:
labeled_df.rank_occurences_by_view(ascending=False).head(20)

                    feat1                  feat2  mean_view_distance
0   contraband_found_rate          driver_gender            0.181818
1   contraband_found_rate            driver_race            0.353208
2   contraband_found_rate                  state            0.241633
3                hit_rate          driver_gender            0.500000
4                hit_rate            driver_race            0.459623
5                hit_rate                  state            0.152449
6               num_stops          driver_gender            0.431818
7               num_stops            driver_race            0.504717
8               num_stops                  state            0.490714
9   search_conducted_rate          driver_gender            0.272727
10  search_conducted_rate            driver_race            0.362736
11  search_conducted_rate              num_stops           78.113680
12  search_conducted_rate                  state            0.323061
13                   year         

,feat1,feat2,group_feat,subgroup,subgroup_trend,subgroup_trend_quality,trend_type,agg_trend,agg_trend_quality,distance,SP_thresh_0.2,mean_view_distance
1546,search_conducted_rate,num_stops,search_conducted_true_num_stops_dpgmm,3,5.36222e+08,0.992416,lin_reg,-257201,0.065594,179.999777,True,78.11368
1494,search_conducted_rate,num_stops,search_conducted_false_search_conducted_true_d...,2,3.36447e+08,1.000000,lin_reg,-257201,0.065594,179.999777,True,78.11368
1511,search_conducted_rate,num_stops,search_conducted_false_contraband_found_false_...,1,3.36447e+08,1.000000,lin_reg,-257201,0.065594,179.999777,True,78.11368
1524,search_conducted_rate,num_stops,search_conducted_false_num_stops_dpgmm,3,3.36447e+08,1.000000,lin_reg,-257201,0.065594,179.999777,True,78.11368
1534,search_conducted_rate,num_stops,search_conducted_true_contraband_found_false_d...,2,3.36447e+08,1.000000,lin_reg,-257201,0.065594,179.999777,True,78.11368
1563,search_conducted_rate,num_stops,contraband_found_false_num_stops_dpgmm,9,3.36447e+08,1.000000,lin_reg,-257201,0.065594,179.999777,True,78.11368
1497,search_conducted_rate,num_stops,search_conducted_false_search_conducted_true_d...,5,3.74615e+07,0.678867,lin_reg,-257201,0.065594,179.999776,True,78.11368
1525,search_conducted_rate,num_stops,search_conducted_false_num_stops_dpgmm,4,3.51834e+07,0.597513,lin_reg,-257201,0.065594,179.999776,True,78.11368
1502,search_conducted_rate,num_stops,search_conducted_false_search_conducted_true_d...,11,3.43021e+07,0.796159,lin_reg,-257201,0.065594,179.999776,True,78.11368
1530,search_conducted_rate,num_stops,search_conducted_false_num_stops_dpgmm,10,3.43021e+07,0.796159,lin_reg,-257201,0.065594,179.999776,True,78.11368


In [26]:
labeled_df.add_view_score('SP_thresh_0.2',agg_type='sum',colored=False)

                    feat1                  feat2  sum_view_SP_thresh_0.2
0   contraband_found_rate          driver_gender                     8.0
1   contraband_found_rate            driver_race                    74.0
2   contraband_found_rate                  state                    50.0
3                hit_rate          driver_gender                    22.0
4                hit_rate            driver_race                    80.0
5                hit_rate                  state                    19.0
6               num_stops          driver_gender                    19.0
7               num_stops            driver_race                    77.0
8               num_stops                  state                    91.0
9   search_conducted_rate          driver_gender                    12.0
10  search_conducted_rate            driver_race                    75.0
11  search_conducted_rate              num_stops                    46.0
12  search_conducted_rate                  state   

,feat1,feat2,group_feat,subgroup,subgroup_trend,subgroup_trend_quality,trend_type,agg_trend,agg_trend_quality,distance,SP_thresh_0.2,mean_view_distance,sum_view_SP_thresh_0.2
0,search_conducted_rate,num_stops,search_conducted_true_num_stops_dpgmm,3,5.36222e+08,0.992416,lin_reg,-257201,0.065594,179.999777,True,78.11368,46.0
1,search_conducted_rate,num_stops,search_conducted_false_search_conducted_true_d...,2,3.36447e+08,1.000000,lin_reg,-257201,0.065594,179.999777,True,78.11368,46.0
2,search_conducted_rate,num_stops,search_conducted_false_contraband_found_false_...,1,3.36447e+08,1.000000,lin_reg,-257201,0.065594,179.999777,True,78.11368,46.0
3,search_conducted_rate,num_stops,search_conducted_false_num_stops_dpgmm,3,3.36447e+08,1.000000,lin_reg,-257201,0.065594,179.999777,True,78.11368,46.0
4,search_conducted_rate,num_stops,search_conducted_true_contraband_found_false_d...,2,3.36447e+08,1.000000,lin_reg,-257201,0.065594,179.999777,True,78.11368,46.0
5,search_conducted_rate,num_stops,contraband_found_false_num_stops_dpgmm,9,3.36447e+08,1.000000,lin_reg,-257201,0.065594,179.999777,True,78.11368,46.0
6,search_conducted_rate,num_stops,search_conducted_false_search_conducted_true_d...,5,3.74615e+07,0.678867,lin_reg,-257201,0.065594,179.999776,True,78.11368,46.0
7,search_conducted_rate,num_stops,search_conducted_false_num_stops_dpgmm,4,3.51834e+07,0.597513,lin_reg,-257201,0.065594,179.999776,True,78.11368,46.0
8,search_conducted_rate,num_stops,search_conducted_false_search_conducted_true_d...,11,3.43021e+07,0.796159,lin_reg,-257201,0.065594,179.999776,True,78.11368,46.0
9,search_conducted_rate,num_stops,search_conducted_false_num_stops_dpgmm,10,3.43021e+07,0.796159,lin_reg,-257201,0.065594,179.999776,True,78.11368,46.0


In [27]:
labeled_df.rank_occurences_by_view('sum_view_SP_thresh_0.2','SP_thresh_0.2').head()

Index(['feat1', 'feat2', 'group_feat', 'subgroup', 'subgroup_trend',
       'subgroup_trend_quality', 'trend_type', 'agg_trend',
       'agg_trend_quality', 'distance', 'SP_thresh_0.2', 'mean_view_distance',
       'sum_view_SP_thresh_0.2'],
      dtype='object')


,feat1,feat2,group_feat,subgroup,subgroup_trend,subgroup_trend_quality,trend_type,agg_trend,agg_trend_quality,distance,SP_thresh_0.2,mean_view_distance,sum_view_SP_thresh_0.2
427,num_stops,state,search_conducted_true_contraband_found_false_d...,11,"[AZ, CT, RI, SC, IL, WA, TX, MD, VT, MA, CO, N...",0.1665,rank_trend,"[RI, VT, WI, CT, MD, MA, AZ, CO, IL, NC, WA, S...",0.3209,1.00,True,0.490714,91.0
428,num_stops,state,search_conducted_false_contraband_found_false_...,11,"[AZ, IL, WA, CT, RI, SC, VT, TX, MD, MA, CO, W...",0.1508,rank_trend,"[RI, VT, WI, CT, MD, MA, AZ, CO, IL, NC, WA, S...",0.3209,0.97,True,0.490714,91.0
429,num_stops,state,hit_ratequantiles,high,"[AZ, IL, CT, SC, NC, TX, RI, VT, MD, WI, CO, MA]",0.3503,rank_trend,"[RI, VT, WI, CT, MD, MA, AZ, CO, IL, NC, WA, S...",0.3209,0.96,True,0.490714,91.0
430,num_stops,state,num_stopsquantiles,mid,"[WI, CT, RI, SC, MA, TX, CO, AZ, NC, WA, MD, I...",0.0469,rank_trend,"[RI, VT, WI, CT, MD, MA, AZ, CO, IL, NC, WA, S...",0.3209,0.87,True,0.490714,91.0
431,num_stops,state,search_conducted_false_search_conducted_true_d...,9,"[AZ, CT, RI, IL, WA, SC, VT, TX, MD, MA, WI, C...",0.2301,rank_trend,"[RI, VT, WI, CT, MD, MA, AZ, CO, IL, NC, WA, S...",0.3209,0.85,True,0.490714,91.0
